In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [79]:
import pandas as pd
data = pd.read_csv('files/Data.csv',usecols = ['name','reviews.text'])
data['product'] = data['name']
data['review'] = data['reviews.text']
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.drop(['name','reviews.text'],axis=1,inplace=True)

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [47]:
llm = ChatOpenAI(temperature=0.9, model='gpt-3.5-turbo')

In [55]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [49]:
chain = LLMChain(llm=llm, prompt=prompt)

In [56]:
product = "Kindle Paperwhite"
chain.run(product)

'Amazon Kindle Paperwhite Manufacturing Co.'

### SimpleSequentialChain

In [60]:
first_prompt = ChatPromptTemplate.from_template(
     "What is the best name to describe \
    a company that makes {product}?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [62]:
second_prompt = ChatPromptTemplate.from_template(
     "write a 20 word description for the follwoig company {company}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [61]:
from langchain.chains import SimpleSequentialChain

In [63]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one,chain_two],verbose=True)

In [65]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
KindleTech
KindleTech is a tech company specializing in e-readers and tablets, offering innovative devices and an extensive library of digital content.

> Finished chain.


'KindleTech is a tech company specializing in e-readers and tablets, offering innovative devices and an extensive library of digital content.'

### SequentialChain

In [75]:
llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")
from langchain.chains import SequentialChain

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [69]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [70]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [71]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [76]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [80]:
review = data.review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'My previous kindle was a DX, this is my second kindle in years. Love the form factor and all but I do miss the physical buttons for page turning. There is a glitch in the software though. I use the English interface but occasionally I would like to translate words into traditional Chinese. However, it seems that the traditional Chinese characters cannot display correctly and become small boxes. The simplified Chinese characters can be displayed correctly though.',
 'English_Review': '我的上一台Kindle是DX，這是多年來的第二台Kindle。喜歡它的外觀和一切，但是我確實想念物理按鈕翻頁的功能。在軟體方面有個小問題。我使用的是英文介面，但偶爾我希望能將詞彙翻譯成繁體中文。然而，繁體中文字無法正確顯示，只會變成小方塊。簡體中文字則可以正確顯示。',
 'summary': 'The reviewer likes the appearance and everything about the Kindle, but misses the physical page-turning buttons and encounters a small issue with displaying traditional Chinese characters correctly.',
 'followup_message': "Thank you for your review of the Kindle. We are thrilled to hear that you are pleased with its appearance and overall performan

### Routerchain

In [81]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [82]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [83]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [85]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

In [87]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [88]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [89]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [90]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [91]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [92]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


C:\Users\iss211\llm\llm-env\Lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue.\n\nBlack body radiation is a fundamental concept in physics and has various applications, including understanding the behavior of stars, explaining the cosmic microwave background radiation, and developing technologies like incandescent light bulbs and thermal imaging cameras.'

In [93]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


"Thank you for your kind words! As a mathematician, I am happy to help with any math questions, no matter how simple or complex they may be.\n\nNow, let's solve the problem at hand. The question is asking for the sum of 2 and 2. To find the answer, we can simply add the two numbers together:\n\n2 + 2 = 4\n\nTherefore, the sum of 2 and 2 is 4."

In [94]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'